# Determine hacking situation using K-Means Clustering & PySpark

These are the features of the hacking data of the company that was attacked:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not.

**The hackers have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks.**

In [0]:
# Start a spark session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('hack').getOrCreate()

In [0]:
# Import data
data=spark.read.csv('dbfs:/FileStore/shared_uploads/hrishagni95@gmail.com/hack_data-1.csv',inferSchema=True,header=True)

In [0]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [0]:
# Import VectorAssembler to assemble the necessary features for clustering
from pyspark.ml.feature import VectorAssembler

In [0]:
# Create the assembler
assembler=VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],outputCol='features')

In [0]:
# Transform the original data with the assembler to get a dense vector of the features
final_data=assembler.transform(data)

In [0]:
# Sneak peek of the transformed data
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed| features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|[8.0,391.09,1.0,2...|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|[20.0,720.99,0.0,...|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|[31.0,356.32,1.0,...|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|[2.0,228.08,1.0,2...|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|[20.0,408.5,0.0,3...|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|[1.0,390.69,1.0,2...|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|[18.0,342.97,1.0,...|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|[22.0,101.61,1.0,...|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|[15.0,275.53,1.0,...|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|[12.0,424.83,1.0,...|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|[15.0,249.09,1.0,...|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|[32.0,242.48,0.0,...|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|[23.0,514.54,0.0,...|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|[9.0,284.77,0.0,3...|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|[27.0,779.25,1.0,...|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|[12.0,307.31,1.0,...|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|[21.0,355.94,1.0,...|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|[10.0,372.65,0.0,...|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|[20.0,347.23,1.0,...|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|[22.0,456.57,0.0,...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
only showing top 20 rows

In [0]:
# Filter only the neccessary column
final_data=final_data.select('features')

In [0]:
# Import StandardScaler to scale the data in 'features', so that the data ranges are scaled to proportion
from pyspark.ml.feature import StandardScaler

In [0]:
# Create an instance of StandardScaler
scale_model=StandardScaler(inputCol='features',outputCol='ScaledFeatures')

In [0]:
# Fit the filtered data on the model
scaled_fit_model=scale_model.fit(final_data)

In [0]:
# Use the fitted model to transform or scale the data
scaled_data=scaled_fit_model.transform(final_data)

In [0]:
# Sneak peek at scaled data
scaled_data.head(3)

Out[19]: [Row(features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), ScaledFeatures=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963])),
 Row(features=DenseVector([20.0, 720.99, 0.0, 3.04, 9.0, 69.08]), ScaledFeatures=DenseVector([1.4196, 2.518, 0.0, 1.3206, 2.9378, 5.151])),
 Row(features=DenseVector([31.0, 356.32, 1.0, 3.71, 8.0, 70.58]), ScaledFeatures=DenseVector([2.2004, 1.2444, 1.9976, 1.6117, 2.6114, 5.2628]))]

In [0]:
# Import Kmeans
from pyspark.ml.clustering import KMeans

In [0]:
# Create an instance of KMeans with k=2, that is with 2 clusters
kmeans=KMeans(featuresCol='ScaledFeatures',k=2).setSeed(1)

In [0]:
# Fit the scaled data
kmeans_model=kmeans.fit(scaled_data)

In [0]:
# Transform the scaled data with the fitted model to get the predictions
pred=kmeans_model.transform(scaled_data)

In [0]:
# Group the predictions to get their count
pred.groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [0]:
# As we can see that the group count matches in number, indicating the analysis made by the forensic team to be true, which in turn concludes that there were 2 hackers and not 3

In [0]:
# Import the Cluster Evaluator to evaluate the model
from pyspark.ml.evaluation import ClusteringEvaluator

In [0]:
silhouette=ClusteringEvaluator(featuresCol='ScaledFeatures')

In [0]:
# Generate the Silhouette score, which is an interpretation and validation of consistency within clusters of data
silhouette.evaluate(pred)

Out[28]: 0.8176460094012482

In [0]:
# Display the cluster coordinates
kmeans_model.clusterCenters()

Out[29]: [array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
 3.28474 ]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
 5.26676612])]